<a href="https://colab.research.google.com/github/mohzary/python-deep-learning-f19/blob/master/M2ICP4/M2ICP4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.tensorflow_backend.set_image_dim_ordering('th')

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [0]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [62]:
num_classes

10

In [0]:
# Create the model
model = Sequential()

#Convolutional input layer, 32 feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
#Dropout layer at 20%.
model.add(Dropout(0.2))
#Convolutional layer, 32 feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
#Max Pool layer with size 2×2.
model.add(MaxPooling2D(pool_size=(2, 2)))
#Convolutional layer, 64 feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
#Dropout layer at 20%.
model.add(Dropout(0.2))
#Convolutional layer, 64 feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
#Max Pool layer with size 2×2
model.add(MaxPooling2D(pool_size=(2, 2)))
#Convolutional layer, 128feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
#Dropout layer at 20%.
model.add(Dropout(0.2))
#Convolutional layer,128 feature maps with a size of 3×3 and a rectifier activation function.
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
#Max Pool layer with size 2×2.
model.add(MaxPooling2D(pool_size=(2, 2)))
#Flatten layer.
model.add(Flatten())
#Dropout layer at 20%.
model.add(Dropout(0.2))
#Fully connected layer with 1024units and a rectifier activation function.
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
#Dropout layerat 20%.
model.add(Dropout(0.2))
#Fully connected layer with 512units and a rectifier activation function.
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
#Dropoutlayer at 20%.
model.add(Dropout(0.2))
#Fully connected output layer with 10 units and a softmax activation function
model.add(Dense(num_classes, activation='softmax'))

In [64]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 16, 16)       

In [65]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-11-16 01:16:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.231.170.111, 54.165.216.26, 34.235.200.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.231.170.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  6.45MB/s    in 2.0s    

2019-11-16 01:17:02 (6.45 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [67]:
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


In [68]:
# Fit the model
from keras.callbacks import TensorBoard
tbCallBack= TensorBoard(log_dir='./Graph', histogram_freq=0,write_graph=True, write_images=True) 
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, callbacks=[tbCallBack])

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 12s 244us/step - loss: 1.8606 - acc: 0.3116 - val_loss: 1.5275 - val_acc: 0.4419
Epoch 2/25
50000/50000 [==============================] - 12s 235us/step - loss: 1.4265 - acc: 0.4795 - val_loss: 1.2870 - val_acc: 0.5368
Epoch 3/25
50000/50000 [==============================] - 12s 236us/step - loss: 1.2331 - acc: 0.5554 - val_loss: 1.1680 - val_acc: 0.5870
Epoch 4/25
50000/50000 [==============================] - 12s 238us/step - loss: 1.0931 - acc: 0.6096 - val_loss: 1.0787 - val_acc: 0.6129
Epoch 5/25
50000/50000 [==============================] - 12s 235us/step - loss: 0.9797 - acc: 0.6517 - val_loss: 0.9347 - val_acc: 0.6717
Epoch 6/25
50000/50000 [==============================] - 12s 243us/step - loss: 0.8915 - acc: 0.6837 - val_loss: 0.8980 - val_acc: 0.6881
Epoch 7/25
50000/50000 [==============================] - 12s 244us/step - loss: 0.8238 - acc: 0.7080 - val_loss: 0.

In [69]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 79.42%


In [73]:
for img in range(0,4):
  predict_classe = model.predict_classes(X_test[[img],:])
  print("predicted classe for image: {" + str(img) +"} is: " + str(predict_classe))


predicted classe for image: {0} is: [3]
predicted classe for image: {1} is: [8]
predicted classe for image: {2} is: [8]
predicted classe for image: {3} is: [0]
